In [1]:
# Siddharth Nagepalli - 210968074
# AI Lab 2 - Ques 1

In [2]:
import csv
import sys

In [3]:
from util import Node, StackFrontier, QueueFrontier

In [4]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [5]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [6]:
def main():
    #if len(sys.argv) > 2:
      #  sys.exit("Usage: python degrees.py [directory]")
        
    #directory = sys.argv[1] if len(sys.argv) == 2 else "large"
     
    directory=input("Enter directory name (small/large):")
    
    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")
        
    print("\n")
        
    print("Solution using bfs : \n")

    path = bfs(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")
            
    print("\n")
            
    print("Solution using dfs: \n")
    
    path1 = dfs(source, target)

    if path1 is None:
        print("Not connected.")
    else:
        degrees1 = len(path1)
        print(f"{degrees1} degrees of separation.")
        path1 = [(None, source)] + path1
        for i in range(degrees1):
            person1 = people[path1[i][1]]["name"]
            person2 = people[path1[i + 1][1]]["name"]
            movie = movies[path1[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")



In [7]:
def bfs(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target, using BFS.

    source and target are unique IMDB actor ID's

    If no possible path, returns None.
    """

    # Initialise a QueueFrontier for BFS, with the starting actor ID:
    start = Node(source, None, None)
    frontier = QueueFrontier()
    frontier.add(start)

    # Initialise an empty explored set to hold explored states (actors):
    explored = set()

    # Loop until a solution is found, or Frontier is empty(no solution):
    while True:

      if len(explored) % 100 == 0:
        print('Actors explored to find solution: ', len(explored))
        print('Nodes left to expand in Frontier: ', len(frontier.frontier))

      # Check for empty Frontier and return with no path if empty
      if frontier.empty():
        print('Frontier is Empty - No Connection Between Actors!')
        print(len(explored), 'actors explored to with no solution found!')
        return None

      # Otherwise expand the next node in the Queue, add it to the explored states and get set of movies and actors for the actor in the current node:
      curr_node = frontier.remove()
      explored.add(curr_node.state)

      for action, state in neighbors_for_person(curr_node.state):

        # If state (actor) is the target actor then solution has been found, return path:
        if state == target:
          print('Solution Found!')
          print(len(explored), 'actors explored to find solution!')
          # Create path from source to target
          path = []
          path.append((action, state))

          # Add action and state to path until back to start node
          while curr_node.parent != None:
            path.append((curr_node.action, curr_node.state))
            curr_node = curr_node.parent

          path.reverse()

          return path

        # Otherwise add the new states to explore to the frontier:
        if not frontier.contains_state(state) and state not in explored:
          new_node = Node(state, curr_node, action)
          frontier.add(new_node)


    # TODO
   # raise NotImplementedError

In [8]:
def dfs(source,target):
    
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target, using DFS.

    source and target are unique IMDB actor ID's

    If no possible path, returns none.
    """
    
    # to keep a track of nodes explored
    num_explored=0
    
    # intialise frontier
    
    start=Node(state=source, parent=None, action=None)
    frontier=StackFrontier()
    frontier.add(start)
    
    # intialise empty explored set
    explored= set()
    
    while True:
        
        if len(explored) % 100 == 0:
            print('Actors explored to find solution: ', len(explored))
            print('Nodes left to expand in Frontier: ', len(frontier.frontier))
        
        # if nothing left in frontier then no path
        if frontier.empty():
            print('Frontier is Empty - No Connection Between Actors!')
            print(len(explored), 'actors explored to with no solution found!')
            return None
            
        # choose node from fronteir
        node = frontier.remove()
        num_explored += 1
        
        # Mark node as explored
        explored.add(node.state)
        
        # add neighbours to frontier
        for movie_id, person_id in neighbors_for_person(node.state):
            if not frontier.contains_state(person_id) and person_id not in explored:
                child = Node(state=person_id, parent=node, action=movie_id)
                
                # if node is the goal , then we have solution
                
                if child.state == target:
                    print('Solution Found!')
                    print(len(explored), 'actors explored to find solution!')
                    movies=[]
                    people=[]
                    solution=[]
                    while child.parent is not None:
                        movies.append(child.action)
                        people.append(child.state)
                        child=child.parent
                    movies.reverse()
                    people.reverse()
                    z=zip(movies,people)
                    for movie, person in z:
                        solution.append((movie,person))
                    return solution
                
                frontier.add(child)
        
    
    
    

In [9]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]



In [10]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


In [11]:

if __name__ == "__main__":
    main()

Enter directory name (small/large):small
Loading data...
Data loaded.
Name: Jack Nicholson
Name: Dustin Hoffman


Solution using bfs : 

Actors explored to find solution:  0
Nodes left to expand in Frontier:  1
Solution Found!
2 actors explored to find solution!
2 degrees of separation.
1: Jack Nicholson and Tom Cruise starred in A Few Good Men
2: Tom Cruise and Dustin Hoffman starred in Rain Man


Solution using dfs: 

Actors explored to find solution:  0
Nodes left to expand in Frontier:  1
Solution Found!
12 actors explored to find solution!
2 degrees of separation.
1: Jack Nicholson and Tom Cruise starred in A Few Good Men
2: Tom Cruise and Dustin Hoffman starred in Rain Man
